### Objective Two

#### Predict stock market volatility using ESG-related news

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import string
import torch
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
from datetime import datetime
import re
import string

# Granger's casuality test library
from statsmodels.tsa.stattools import grangercausalitytests

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


# Import VADER for sentiment analysis
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /Users/luca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/luca/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Step 1: Calculate the daily market returns time series and volatility

In [2]:
# Load price data
prices_file_path = '../Data/Input/Eikon/refinitiv_prices_raw.csv'


# Read the CSV file into a DataFrame
prices_df = pd.read_csv(prices_file_path)

prices_df

,company,symbol,date,close,open,high,low
0,Ford,F,2024-04-01,13.29,13.33,13.380,13.140
1,Ford,F,2024-04-02,13.28,13.16,13.370,13.090
2,Ford,F,2024-04-03,13.65,13.25,13.680,13.230
3,Ford,F,2024-04-04,13.21,13.90,13.950,13.170
4,Ford,F,2024-04-05,13.28,13.27,13.395,13.090
...,...,...,...,...,...,...,...
955,Tesco,TSCO.L,2024-08-09,330.50,328.50,331.100,326.200
956,Tesco,TSCO.L,2024-08-12,333.50,331.70,333.900,330.880
957,Tesco,TSCO.L,2024-08-13,335.00,333.80,338.000,333.400
958,Tesco,TSCO.L,2024-08-14,340.60,336.50,340.600,336.400


In [3]:
# Let's filter the dataset to match the dates of the available news stories 

# Ensure 'date' column is in datetime format
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Define the date range for filtering
start_date = '2024-05-01'
end_date = '2024-07-31'

# Filter the DataFrame to keep rows between 1st May 2024 and 31st July 2024
filtered_df = prices_df[(prices_df['date'] >= start_date) & (prices_df['date'] <= end_date)]

# Display the filtered data
print(filtered_df.head())


   company symbol       date  close   open   high      low
22    Ford      F 2024-05-01  12.20  12.16  12.43  12.1500
23    Ford      F 2024-05-02  12.49  12.40  12.55  12.3600
24    Ford      F 2024-05-03  12.43  12.64  12.76  12.3900
25    Ford      F 2024-05-06  12.50  12.54  12.62  12.4575
26    Ford      F 2024-05-07  12.17  12.40  12.45  12.0850


In [4]:
filtered_df

,company,symbol,date,close,open,high,low
22,Ford,F,2024-05-01,12.20,12.16,12.4300,12.1500
23,Ford,F,2024-05-02,12.49,12.40,12.5500,12.3600
24,Ford,F,2024-05-03,12.43,12.64,12.7600,12.3900
25,Ford,F,2024-05-06,12.50,12.54,12.6200,12.4575
26,Ford,F,2024-05-07,12.17,12.40,12.4500,12.0850
...,...,...,...,...,...,...,...
944,Tesco,TSCO.L,2024-07-25,325.60,325.60,327.1000,323.4000
945,Tesco,TSCO.L,2024-07-26,327.10,325.10,328.6000,323.3490
946,Tesco,TSCO.L,2024-07-29,330.90,330.00,333.7000,328.1630
947,Tesco,TSCO.L,2024-07-30,331.70,330.00,332.8000,328.4000


The daily market return on day t is calculated as:

\begin{equation}
r_{t} = ln \left(\frac{CLOSE_{t}}{CLOSE_{t-1}}\right)
\end{equation}

where $CLOSE_{t}$ is the closing price on day t and $CLOSE_{t-1}$ is the previous day closing price.

In [5]:
# Sort by company and date to maintain the proper order for each company
filtered_df = filtered_df.sort_values(by=['company', 'date'])


In [6]:
# Calculate daily market return using the formula: r_t = log(CLOSE_t / CLOSE_t-1)
filtered_df['daily_return'] = filtered_df.groupby('company', group_keys=False)['close'].apply(
    lambda x: np.log(x / x.shift(1))
)

# Display the result
print(filtered_df.head())

    company symbol       date  close   open    high    low  daily_return
502    Asda    WMT 2024-05-01  58.85  59.31  59.410  58.72           NaN
503    Asda    WMT 2024-05-02  59.71  58.94  59.885  58.58      0.014508
504    Asda    WMT 2024-05-03  59.82  59.62  59.980  59.14      0.001841
505    Asda    WMT 2024-05-06  59.87  60.00  60.000  59.39      0.000835
506    Asda    WMT 2024-05-07  60.62  60.17  60.800  60.05      0.012449


In [7]:
filtered_df

,company,symbol,date,close,open,high,low,daily_return
502,Asda,WMT,2024-05-01,58.85,59.31,59.410,58.7200,NaN
503,Asda,WMT,2024-05-02,59.71,58.94,59.885,58.5800,0.014508
504,Asda,WMT,2024-05-03,59.82,59.62,59.980,59.1400,0.001841
505,Asda,WMT,2024-05-06,59.87,60.00,60.000,59.3900,0.000835
506,Asda,WMT,2024-05-07,60.62,60.17,60.800,60.0500,0.012449
...,...,...,...,...,...,...,...,...
464,Toyota,TM,2024-07-25,195.25,197.43,197.430,193.7300,-0.009938
465,Toyota,TM,2024-07-26,192.52,190.74,192.840,190.5100,-0.014081
466,Toyota,TM,2024-07-29,192.48,193.00,193.200,191.8067,-0.000208
467,Toyota,TM,2024-07-30,193.11,194.96,195.480,192.2650,0.003268


Calculate time series Volatility

The volatility of the stock market index is calculated within a defined time window (e.g., previous 90 days):

\begin{equation}
Vol = \sqrt{\frac{1}{N}\sum_{t=1}^{N}(r_{t}-\bar{r})^2} \cdot \sqrt{252}
\end{equation}

where N is the total number of days during a window time of observations (eg, 30 days), and 252 is the total number of trading days in a single year;

In [8]:
def calculate_volatility(df):
    # Group by company to calculate volatility for each stock
    volatility_df = df.groupby('company').apply(lambda x: calculate_stock_volatility(x))
    
    # Reset the index 
    volatility_df = volatility_df.reset_index(drop=True)
    
    return volatility_df

def calculate_stock_volatility(stock_df):
    # Number of trading days (rows)
    N = len(stock_df)
    
    # Mean of daily returns (r̄)
    mean_return = stock_df['daily_return'].mean()
    
    # Variance calculation: (r_t - r̄)^2
    variance = np.sum((stock_df['daily_return'] - mean_return) ** 2) / N
    
    # Daily volatility: sqrt(variance)
    daily_volatility = np.sqrt(variance)
    
    # Annual volatility: daily_volatility * sqrt(252)
    annual_volatility = daily_volatility * np.sqrt(252)
    
    # Return the company and its calculated volatility
    return pd.Series({'company': stock_df['company'].iloc[0], 'volatility': annual_volatility})


# Assuming filtered_df is already loaded and contains 'company' and 'daily_return' columns
volatility_results = calculate_volatility(filtered_df)

# Display the calculated volatility for each company
print(volatility_results)


           company  volatility
0             Asda    0.187419
1             Ford    0.477228
2  Marks & Spencer    0.250589
3            Ocado    0.745336
4         Polestar    1.047160
5       Sainsburys    0.195476
6       Stellantis    0.294795
7            Tesco    0.120855
8            Tesla    0.566548
9           Toyota    0.224128


In [9]:
# Sort the DataFrame by 'volatility' in descending order
volatility_results = volatility_results.sort_values(by='volatility', ascending=False)
volatility_results

,company,volatility
4,Polestar,1.047160
3,Ocado,0.745336
8,Tesla,0.566548
1,Ford,0.477228
6,Stellantis,0.294795
2,Marks & Spencer,0.250589
9,Toyota,0.224128
5,Sainsburys,0.195476
0,Asda,0.187419
7,Tesco,0.120855


### Step 2: Run sentiment Analysis of the ESG news stories using VEDER and set up the daily sentiment score time series

In [10]:
# Load Pre-Processed ESG stories data from Objective One
stories_file_path = '../Data/Output/news_df.csv'


# Read the CSV file into a DataFrame
news_df = pd.read_csv(stories_file_path)

news_df

,story,date,company,ticker
0,los angeles ca accesswire july 29 2024 the ...,2024-07-29,Ford,esg
1,new york city ny accesswire july 29 2024 br...,2024-07-29,Ford,esg
2,ford alert bragar eagel amp squire pc is inves...,2024-07-29,Ford,esg
3,first atlantic nickel corp fanv\nalaska energy...,2024-07-29,Ford,esg
4,palm beach fla july 29 2024 globe newswire ...,2024-07-29,Ford,esg
...,...,...,...,...
985,tesco the uks largest supermarket chain has sp...,2024-05-15,Tesco,governance
986,tesco has been accused of giving struggling wo...,2024-05-14,Tesco,governance
987,tesco boss ken murphy has seen his pay deal mo...,2024-05-14,Tesco,governance
988,tesco has apologised after a black publisher s...,2024-05-20,Tesco,sustainability


In [11]:
# Let's initialise the 'stop words' function for English 

stop = stopwords.words('english')
stop[:5]

['i', 'me', 'my', 'myself', 'we']

In [12]:
# Let's remove 'stop words' from the stories

news_df['story'] = news_df['story'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop]))
news_df.head()

,story,date,company,ticker
0,los angeles ca accesswire july 29 2024 schall ...,2024-07-29,Ford,esg
1,new york city ny accesswire july 29 2024 brons...,2024-07-29,Ford,esg
2,ford alert bragar eagel amp squire pc investig...,2024-07-29,Ford,esg
3,first atlantic nickel corp fanv alaska energy ...,2024-07-29,Ford,esg
4,palm beach fla july 29 2024 globe newswire fin...,2024-07-29,Ford,esg


In [13]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to apply VADER and get sentiment scores
def apply_vader_sentiment(text):
    # Get the sentiment scores from VADER
    sentiment_scores = analyzer.polarity_scores(text)
    return sentiment_scores

# Apply VADER sentiment analysis to the "story" column
news_df['vader_sentiment'] = news_df['story'].apply(apply_vader_sentiment)

# Split the sentiment scores into separate columns (optional)
news_df = pd.concat([news_df.drop(['vader_sentiment'], axis=1), news_df['vader_sentiment'].apply(pd.Series)], axis=1)

# Define a function to classify sentiment based on the 'compound' score
def classify_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the classify_sentiment function to create a new 'sentiment' column
news_df['sentiment'] = news_df['compound'].apply(classify_sentiment)

news_df

,story,date,company,ticker,neg,neu,pos,compound,sentiment
0,los angeles ca accesswire july 29 2024 schall ...,2024-07-29,Ford,esg,0.100,0.779,0.121,0.5267,positive
1,new york city ny accesswire july 29 2024 brons...,2024-07-29,Ford,esg,0.065,0.864,0.071,-0.1531,negative
2,ford alert bragar eagel amp squire pc investig...,2024-07-29,Ford,esg,0.063,0.800,0.137,0.9442,positive
3,first atlantic nickel corp fanv alaska energy ...,2024-07-29,Ford,esg,0.033,0.818,0.149,0.9995,positive
4,palm beach fla july 29 2024 globe newswire fin...,2024-07-29,Ford,esg,0.033,0.819,0.148,0.9994,positive
...,...,...,...,...,...,...,...,...,...
985,tesco uks largest supermarket chain sparked co...,2024-05-15,Tesco,governance,0.123,0.696,0.181,0.9442,positive
986,tesco accused giving struggling workers slap f...,2024-05-14,Tesco,governance,0.116,0.704,0.180,0.9863,positive
987,tesco boss ken murphy seen pay deal double alm...,2024-05-14,Tesco,governance,0.057,0.747,0.196,0.9882,positive
988,tesco apologised black publisher says racially...,2024-05-20,Tesco,sustainability,0.142,0.732,0.125,-0.8141,negative


In [ ]:
news_df

In [14]:
# Function to normalise sentiment proportions and calculate Sent_d
def calculate_sentiment_score(df):
    # Calculate total count of all sentiment categories
    df['total'] = df['pos'] + df['neu'] + df['neg']
    
    # Normalize to get probabilities (frequencies) of positive, neutral, and negative
    df['pos_prob'] = df['pos'] / df['total']
    df['neut_prob'] = df['neu'] / df['total']
    df['neg_prob'] = df['neg'] / df['total']
    
    # Confirm the probabilities sum to 1
    df['sum_probs'] = df['pos_prob'] + df['neut_prob'] + df['neg_prob']
    
    # Calculate Sent_d using the normalized probabilities
    df['Sent_d'] = (df['pos'] - df['neg']) / (df['pos'] + df['neu'] + df['neg'] + 3)
    
    return df


# Group by company and date to ensure daily aggregation
grouped_df = news_df.groupby(['company', 'date']).sum().reset_index()

# Apply the sentiment calculation
result_df = calculate_sentiment_score(grouped_df)

# Show the resulting DataFrame with normalised probabilities and Sent_d
print(result_df[['company', 'date', 'pos_prob', 'neut_prob', 'neg_prob', 'sum_probs', 'Sent_d']])


    company        date  pos_prob  neut_prob  neg_prob  sum_probs    Sent_d
0      Asda  2024-06-07  0.135000   0.770000  0.095000        1.0  0.010000
1      Asda  2024-07-08  0.139000   0.852000  0.009000        1.0  0.052000
2      Asda  2024-07-23  0.093000   0.640000  0.267000        1.0 -0.043500
3      Asda  2024-07-24  0.154845   0.804196  0.040959        1.0  0.045582
4      Asda  2024-07-26  0.094000   0.621000  0.285000        1.0 -0.047750
..      ...         ...       ...        ...       ...        ...       ...
211  Toyota  2024-07-25  0.126452   0.774171  0.099377        1.0  0.025141
212  Toyota  2024-07-26  0.128203   0.786056  0.085742        1.0  0.040057
213  Toyota  2024-07-27  0.154125   0.768875  0.077000        1.0  0.056091
214  Toyota  2024-07-28  0.142884   0.749781  0.107335        1.0  0.029938
215  Toyota  2024-07-29  0.137161   0.745200  0.117638        1.0  0.018161

[216 rows x 7 columns]


In [15]:
result_df

,company,date,story,ticker,neg,neu,pos,compound,sentiment,total,pos_prob,neut_prob,neg_prob,sum_probs,Sent_d
0,Asda,2024-06-07,zuber issa ended partnership asda brother mohs...,esg,0.095,0.770,0.135,0.9432,positive,1.000,0.135000,0.770000,0.095000,1.0,0.010000
1,Asda,2024-07-08,asda released 2023 brighter living report deta...,esgsustainability,0.018,1.704,0.278,1.9780,positivepositive,2.000,0.139000,0.852000,0.009000,1.0,0.052000
2,Asda,2024-07-23,150 members gmb union participate strike 2 aug...,esg,0.267,0.640,0.093,-0.9911,negative,1.000,0.093000,0.640000,0.267000,1.0,-0.043500
3,Asda,2024-07-24,abha july 24 2024 spa national center vegetati...,environmentsustainability,0.082,1.610,0.310,1.8520,positivepositive,2.002,0.154845,0.804196,0.040959,1.0,0.045582
4,Asda,2024-07-26,asda supermarket workers wisbech poised anothe...,esg,0.285,0.621,0.094,-0.9931,negative,1.000,0.094000,0.621000,0.285000,1.0,-0.047750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Toyota,2024-07-25,new york ny accesswire july 25 2024 suffered l...,esgesgesgesgesgesgesgesgesgesgesgesgesgsocials...,3.876,30.195,4.932,7.2438,positivenegativepositivepositivepositivepositi...,39.003,0.126452,0.774171,0.099377,1.0,0.025141
212,Toyota,2024-07-26,new york ny accesswire july 26 2024 suffered l...,esgesgesgesgesgesgesgesgesgesgesgesgesgesgesge...,4.287,39.302,6.410,18.6472,positivepositivepositivepositivenegativepositi...,49.999,0.128203,0.786056,0.085742,1.0,0.040057
213,Toyota,2024-07-27,new york new yorknewsfile corp july 27 2024 ro...,esgesgsocialsocialsocialsocialgovernancegovern...,0.616,6.151,1.233,5.9395,positivepositivepositivepositivepositivenegati...,8.000,0.154125,0.768875,0.077000,1.0,0.056091
214,Toyota,2024-07-28,new york ny accesswire july 28 2024 suffered l...,esgesgesgesgesgesgsocialsocialsocialsocialsoci...,1.718,12.001,2.287,5.5345,positivepositivepositivepositivenegativepositi...,16.006,0.142884,0.749781,0.107335,1.0,0.029938


Step 3: perform the Granger’s causality testing using the dedicated Python library “grangercausalitytests.” 

Step 4: The Latent Dirichlet Allocation (LDA) model will be implemented
using dedicated Python libraries such as Gensim which is a library for topic modeling and natural language processing. Below are the indicative steps that will be
followed to build an LDA model:
- Create a dataset containing news stories;
- Perform text preprocessing by tokenising and cleaning the text;
- Generate a few LDA models using different topic values, then verify how
these models perform in the supervised classification model training
(Kelechava 2019);
- Visualise the topics generated with the words associated with each topic,
and
- Transform the original texts (the headlines) to the topic vectors.

Step 5: Finally, the topic vectors will be fed into a classifier and the process validated
by splitting the topic dataframe into train and test to simulate how the model
would perform with a new set of data. The classification report will be generated
to evaluate the model.